In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
FOLDER_PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\scirep-cuisines-detail"
FILES = ["allr_recipes.txt","epic_recipes.txt","menu_recipes.txt"]

In [3]:
recipes = []
for file in FILES:
    with open(os.path.join(FOLDER_PATH,file)) as f:
        for i,recipe in enumerate(f):
            data = recipe.strip('\n').split('\t')
            recipes.append({"cuisine":data[0],"id":i,"ingredients":data[1:]})
        
df_train = pd.DataFrame(recipes)
df_train['all_ingredients'] = df_train['ingredients'].map(";".join)

### Data cleaning

In [4]:
# Fusion of the same cuisines
df_train=df_train.replace({"Mexico":"Mexican",
                           "mexico":"Mexican",
                           "chinese":"Chinese",
                           "China":"Chinese",
                           "France":"French",
                           "japanese":"Japanese",
                           "Japan":"Japanese",
                           "Thailand":"Thai",
                           "German":"Germany",
                           "India":"Indian",
                           "Israel":"Jewish",
                           "italian":"Italian",
                           "Italy":"Italian",
                           "Scandinavia":"Scandinavian",
                           "Vietnam":"Vietnamese",
                           "Korea":"Korean",
                           "korean":"Korean",
                           "EasternEuropean_Russian":"Eastern-Europe",
                           'Spain':'Spanish_Portuguese'})
df_train

,cuisine,id,ingredients,all_ingredients
0,Canada,0,"[egg, yeast, wheat, milk, lard]",egg;yeast;wheat;milk;lard
1,Canada,1,"[pork, carrot, pea, onion, potato]",pork;carrot;pea;onion;potato
2,Canada,2,[maple_syrup],maple_syrup
3,Canada,3,"[wheat, yeast, almond, honey, oat, date, veget...",wheat;yeast;almond;honey;oat;date;vegetable_oi...
4,Canada,4,"[butter, lovage, clam, wheat, onion, thyme, po...",butter;lovage;clam;wheat;onion;thyme;potato;ye...
...,...,...,...,...
57686,Italian,2453,"[kiwi, olive_oil, clam, white_wine, orange, sa...",kiwi;olive_oil;clam;white_wine;orange;salmon;f...
57687,Italian,2454,"[tomato, butter, beef, onion, red_wine, black_...",tomato;butter;beef;onion;red_wine;black_pepper...
57688,Italian,2455,"[vegetable, wheat, egg, cheese, olive_oil]",vegetable;wheat;egg;cheese;olive_oil
57689,Italian,2456,"[tomato, clam, black_pepper, parsley, celery, ...",tomato;clam;black_pepper;parsley;celery;macaroni


## Removing cuisines with not enough recipes

In [5]:
RECIPE_THRESHOLD=10
cuisine_count= df_train['cuisine'].value_counts()
to_drop = [cuisine_count[cuisine_count == el].index[0] for el in cuisine_count if el<RECIPE_THRESHOLD]
df_train = df_train[~df_train["cuisine"].isin(to_drop)]

In [6]:
# Saving the cleaned data
df_train.to_pickle(os.path.join(FOLDER_PATH,"full_data.pkl"))